In [1]:
# Importing the libraries

import torch as th
import torch as tn
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import torchtext

from torchtext import vocab
import pickle as pl
from tqdm import tqdm
import numpy as np

/Users/magnusharder/Documents/UNI-DTU/6. Semester/Arkitektur - Special Course/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
d_model = 128
dim_input = d_model
dim_recurrent = d_model
num_layers = 27
dim_output = tgt_vocab_size
src_padding_idx = Vocab_en.__getitem__('<Pad>')



LSTM = LSTMNet(
    dim_input = dim_input,
    dim_recurrent = dim_recurrent,
    num_layers = num_layers,
    dim_output = dim_output,
    src_padding_idx = src_padding_idx
)

GRU = GRUNet(
    dim_input = dim_input,
    dim_recurrent = dim_recurrent,
    num_layers = num_layers,
    dim_output = dim_output,
    src_padding_idx = src_padding_idx
)

In [24]:
# Define Training Function
def train(model, optimizer, loss_fn, n_epochs, n_batches, X_train_batches, Y_train_batches, src_padding_idx):
    for epoch in range(n_epochs):
        loss_epoch = 0
        for batch in range(n_batches):
            # Get the data
            X_batch = X_train_batches[batch]
            Y_batch = Y_train_batches[batch]

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            y_pred = model(X_batch)

            # Compute the loss
            loss = 0 
            for sample in range(batch_size):
                loss += loss_fn(y_pred[sample], Y_batch[sample])

            # Backward pass
            loss.backward()

            # Update the parameters
            optimizer.step()

            # Print the loss
            loss_epoch += loss.item()

        print('Epoch: %d, Batch: %d, Loss: %f' % (epoch, batch, loss_epoch/n_train))

In [29]:
# Define the loss function
loss_fn = nn.CrossEntropyLoss()

# Define the optimizer LSTM
optimizer_LSTM = th.optim.Adam(LSTM.parameters(), lr=0.001)

# Define the optimizer GRU
optimizer_GRU = th.optim.Adam(GRU.parameters(), lr=0.001)

# Define the number of epochs
n_epochs = 1000

# Define the number of batches
n_batches = 20

#train(LSTM, optimizer_LSTM, loss_fn, n_epochs, n_batches, X_train_batches, Y_train_batches, src_padding_idx)

In [30]:
train(GRU, optimizer_GRU, loss_fn, n_epochs, n_batches, X_train_batches, Y_train_batches, src_padding_idx)

tn.save(GRU.state_dict(), "GRU.pt")

Epoch: 0, Batch: 19, Loss: 2.224160
Epoch: 1, Batch: 19, Loss: 2.156827
Epoch: 2, Batch: 19, Loss: 2.115526
Epoch: 3, Batch: 19, Loss: 2.077500
Epoch: 4, Batch: 19, Loss: 2.047868
Epoch: 5, Batch: 19, Loss: 2.016982
Epoch: 6, Batch: 19, Loss: 1.988233
Epoch: 7, Batch: 19, Loss: 1.964263
Epoch: 8, Batch: 19, Loss: 1.938313
Epoch: 9, Batch: 19, Loss: 1.916509
Epoch: 10, Batch: 19, Loss: 1.898295
Epoch: 11, Batch: 19, Loss: 1.877218
Epoch: 12, Batch: 19, Loss: 1.855456
Epoch: 13, Batch: 19, Loss: 1.836794
Epoch: 14, Batch: 19, Loss: 1.820911
Epoch: 15, Batch: 19, Loss: 1.807434
Epoch: 16, Batch: 19, Loss: 1.794196
Epoch: 17, Batch: 19, Loss: 1.781334
Epoch: 18, Batch: 19, Loss: 1.768077
Epoch: 19, Batch: 19, Loss: 1.754433
Epoch: 20, Batch: 19, Loss: 1.747602
Epoch: 21, Batch: 19, Loss: 1.737926
Epoch: 22, Batch: 19, Loss: 1.728831
Epoch: 23, Batch: 19, Loss: 1.721961
Epoch: 24, Batch: 19, Loss: 1.715296
Epoch: 25, Batch: 19, Loss: 1.709039
Epoch: 26, Batch: 19, Loss: 1.703716
Epoch: 27, 

In [27]:
out = LSTM(X_train_batches[0][0])
out.argmax(dim = 1)

tensor([206, 206,   0,   0,   5,   5,   1,   1,   1, 205, 205, 205, 205, 205,
        205, 205, 205, 205, 205, 205, 205, 205, 205, 205, 205, 205, 205])

In [32]:
out = GRU(X_test[0])
out.argmax(dim = 1)

tensor([206,   6,   0,   0,   1,   5,   1,   1,   7,   4,   0, 205, 205,   3,
        205, 205, 205, 205, 205, 205, 205, 205, 205, 205, 205, 205, 205])

In [33]:
Y_test[0]

tensor([206,  31,  13,  12,  14,   0,   6,  84,   1,   7,  30,  13,  12,   0,
          6,  87,   2, 204, 205, 205, 205, 205, 205, 205, 205, 205, 205])